<h5>This code is part of XLR. It implements cosine decay learning rate model training. Date: 01/09/2025</h5>

<h5>Contact: rakibul.haque@utsa.edu</h5>

<h5>Cite as: R. U. Haque and P. Markopoulos,"XLR: A Universal Framework for Learning rate Adaptation via Exponential Range Exploration", 2025</h5>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from collections import Counter
import random
import copy
import torch.nn.functional as F
import sys
import time
import pickle
import pandas as pd
import random
import queue
from collections import deque
from torch import Tensor
from typing import Type
import torchvision.models as models
import math
import matplotlib.pyplot as plt
from torch.utils.data import random_split, DataLoader
import math
import gc
import os

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [3]:
def Print(string, dictionary):
    first_key = next(iter(dictionary))
    first_value = dictionary[first_key]
    print(f"{string}:{first_key}: {first_value[0][0]}\n")

In [4]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # Change output to 10 for CIFAR-10 classes

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.relu5(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

In [5]:
def evaluate(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    # Compute average loss and accuracy
    test_loss = running_loss / len(test_loader)
    test_accuracy = 100 * correct / total

    return test_loss, test_accuracy

In [6]:
# Initialize model
model_centralized = cnn()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_centralized.to(device)

file_path = "s_cnn.pth"
initial_weights=torch.load(file_path, weights_only=True)
Print("Model's initial weights", initial_weights)

model_centralized.load_state_dict(initial_weights)

# b_size=128
# num_epochs = 50
# le_rate=0.01

# Hyperparameters
batch_size = 128
num_epochs = 50
eta_min = 0.00001
eta_max = 0.1
rho_w = 10
ntest = 10
confidence = 0.99
patience = 2

opti="sgd"
# opti="adam"
# opti="adagrad"
# opti="adadelta"
# opti="rmdprop"

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model_centralized.parameters(), lr=0.01)

if opti=="sgd":
    optimizer = torch.optim.SGD(model_centralized.parameters(), lr=eta_min)
elif opti=="adam":
    optimizer = torch.optim.Adam(model_centralized.parameters(), lr=eta_min)
elif opti=="adagrad":
    optimizer = torch.optim.Adagrad(model_centralized.parameters(), lr=eta_min)
elif opti=="adadelta":
    optimizer = torch.optim.Adadelta(model_centralized.parameters(), lr=eta_min)
elif opti=="rmdprop":
    optimizer = torch.optim.RMSprop(model_centralized.parameters(), lr=eta_min)


name=f"==related_work_large_batchsize_{opti}_{batch_size}_{num_epochs}"

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
val_losses = []
val_accuracies = []
learning_rate= []
learning_rate.append(None)



# DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


subset_dataset = torch.load('val_dataset.pth',weights_only=False)
remaining_dataset = torch.load('test_dataset.pth',weights_only=False)
val_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(remaining_dataset, batch_size=batch_size, shuffle=False)


# Learning rate scheduler
total_steps = len(train_loader) * num_epochs
warmup_steps = int(rho_w * total_steps)
gamma = (eta_max / eta_min) ** (1 / warmup_steps)


# AutoWU Implementation
lr = eta_min
patience_flag = 0
warmup_phase = True
loss_history = []

Model's initial weights:conv1.weight: tensor([[ 0.1329, -0.1625,  0.0656],
        [ 0.0480,  0.0967,  0.0128],
        [ 0.1696,  0.0517, -0.0104]], device='cuda:0')



In [7]:
def compute_loss_and_gradient(model, optimizer, inputs, labels):
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def cosine_decay_lr(step, T, eta_min, eta_max):
    return eta_min + 0.5 * (eta_max - eta_min) * (1 + np.cos(np.pi * step / T))

In [8]:
#round zero
train_loss, train_accuracy = evaluate(model_centralized, train_loader, criterion, device)
train_losses.append(train_loss)
train_accuracies.append(train_accuracy)

val_loss, val_accuracy = evaluate(model_centralized, val_loader, criterion, device)
val_losses.append(val_loss)
val_accuracies.append(val_accuracy)

test_loss, test_accuracy = evaluate(model_centralized, test_loader, criterion, device)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

In [9]:
step_counter = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_corrects = 0  # Tracks the number of correct predictions
    running_loss=0
    total_samples = 0  # Tracks the total number of samples processed
    
    for inputs, labels in train_loader:
        
        inputs, labels = inputs.to(device), labels.to(device)
        # Increment the step counter
        step_counter += 1
        
        # Forward pass
        outputs = model_centralized(inputs)
        _, preds = torch.max(outputs, 1)  # Get predictions
        total_samples += labels.size(0)
        running_corrects += torch.sum(preds == labels.data).item()
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss
        
        # Update learning rate during warmup phase
        if warmup_phase:
            lr = lr * gamma if step_counter < warmup_steps else lr
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            
            # End of warmup phase checks
            if step_counter >= warmup_steps:
                warmup_phase = False
                # Implement Gaussian Process and confidence checks as needed
        else:
            # Cosine decay after warmup
            lr = cosine_decay_lr(step_counter - warmup_steps, total_steps - warmup_steps, eta_min, eta_max)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
    
    # Calculate training accuracy for the epoch
    train_accuracy = running_corrects / total_samples * 100
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    learning_rate.append(optimizer.param_groups[0]['lr'])

    #===========================================================================================
    # Evaluate the model and get test loss and accuracy
    val_loss, val_accuracy = evaluate(model_centralized, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    test_loss, test_accuracy = evaluate(model_centralized, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Epoch 1/50
Train Loss: 2.3027, Train Accuracy: 9.99%
Test Loss: 2.3027, Test Accuracy: 10.01%
Epoch 2/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 3/50
Train Loss: 2.3027, Train Accuracy: 10.01%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 4/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 5/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 6/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 7/50
Train Loss: 2.3027, Train Accuracy: 10.01%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 8/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 9/50
Train Loss: 2.3027, Train Accuracy: 10.00%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 10/50
Train Loss: 2.3027, Train Accuracy: 10.02%
Test Loss: 2.3026, Test Accuracy: 10.01%
Epoch 11/50
Train Loss: 2.3027, Train Accuracy: 10

In [13]:
# Convert tensor data to CPU and then to Python lists if necessary
train_losses = [loss.item() if torch.is_tensor(loss) else loss for loss in train_losses]
train_accuracies = [accuracy.item() if torch.is_tensor(accuracy) else accuracy for accuracy in train_accuracies]
val_losses = [loss.item() if torch.is_tensor(loss) else loss for loss in val_losses]
val_accuracies = [accuracy.item() if torch.is_tensor(accuracy) else accuracy for accuracy in val_accuracies]
test_losses = [loss.item() if torch.is_tensor(loss) else loss for loss in test_losses]
test_accuracies = [accuracy.item() if torch.is_tensor(accuracy) else accuracy for accuracy in test_accuracies]
learning_rate = [lr.item() if torch.is_tensor(lr) else lr for lr in learning_rate]

# # Ensure the directory exists
os.makedirs(name, exist_ok=True)
# Prepare data for DataFrame
data = {
    "Train Losses": train_losses,
    "Train Accuracies": train_accuracies,
    "Validation Losses": val_losses,
    "Validation Accuracies": val_accuracies,
    "Test Losses": test_losses,
    "Test Accuracies": test_accuracies,
    "Learning Rate": learning_rate,
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
file_path = f'{name}/{name}_metrics.csv'
df.to_csv(file_path, index=False)